# River Tree

An example workflow for gathering NHD Plus data and generating a tree.

In [1]:
import os,sys
sys.path.append('/Users/ajc/Core/SimDataInputs/WorkSpace/ats-88/ats_meshing/workflow/')
import numpy as np
from matplotlib import pyplot as plt
import shapely
import logging
sys.path.append('/Users/ajc/anaconda3/envs/ats_meshing/')
import workflow
import workflow.source_list
import workflow.conf
import workflow.ui
import workflow.utils
import workflow.plot
workflow.conf.rcParams['data dir'] = os.path.join(os.getcwd(),'..','data')

workflow.ui.setup_logging(1,None)

In [2]:
# open a shapefile for use here
#use fiona to read shape file
manager_shp = workflow.source_list.FileManagerShape('../data/hydrologic_units/others/coweeta_basin/coweeta_basin.shp')
shp_profile,shp = manager_shp.get_shape()
shply = workflow.utils.shply(shp)

In [3]:
# find the rivers in this shape
#call to hilev.py
reaches, _ = workflow.get_rivers_by_bounds(workflow.source_list.hydrography_sources['NHD Plus'],
                                               shply.bounds, shp_profile['crs'], '0601', merge=False)

# filter the list to only those that intersect the shape
reaches = workflow.hydrography.filter_rivers_to_shape(shply, reaches, 10)

# make the global tree
rivers = workflow.hydrography.make_global_tree(reaches)

# check that only one tree was formed (this means al)
assert(len(rivers) is 1)
river = rivers[0]


2019-10-16 15:33:08,085 - root - INFO: 
2019-10-16 15:33:08,086 - root - INFO: Preprocessing Hydrography
2019-10-16 15:33:08,087 - root - INFO: ------------------------------
2019-10-16 15:33:08,087 - root - INFO: loading streams in bounds (273971.0911428096, 3878839.6361173145, 279140.9150949494, 3883953.7853134344)
2019-10-16 15:33:08,089 - root - INFO: Using Hydrography file "/Users/ajc/Core/SimDataInputs/WorkSpace/ats-88/ats_meshing/examples/../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb"
2019-10-16 15:33:09,779 - root - INFO:   ...filtering


In [6]:
%matplotlib
HSV_tuples=['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', 
            '#fabebe', '#008080', '#e6beff', '#9a6324', '#a9a9a9', '#800000', '#aaffc3', '#808000', '#ffd8b1', 
            '#000075', '#808080', '#000000', '#009090', '#090770']
workflow.plot.rivers([river,], shp_profile['crs'],color=HSV_tuples)


Using matplotlib backend: MacOSX


ConnectionError: HTTPConnectionPool(host='epsg.io', port=80): Max retries exceeded with url: /26917.gml?download (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1155c0d68>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [ ]:
#import colorsys
#N = 22
#HSV_tuples = [(x*1.0/N, x*.1/N, 1.0) for x in range(N)]
#RGB_tuples = map(lambda x: colorsys.hsv_to_rgb(*x), HSV_tuples)

In [ ]:
river_NHDPlusID = []
for r in river:
    print (r,r.properties)
    river_NHDPlusID.append(r.properties['NHDPlusID'])    

#print (river_NHDPlusID)

In [ ]:
import fiona
fiona.listlayers('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb')

In [ ]:
FlowlineVAA = fiona.open('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb', 'r', layer='NHDPlusFlowlineVAA')
Flowline = fiona.open('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb', 'r', layer='NHDFlowline')
Flow = fiona.open('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb', 'r', layer='NHDPlusFlow')
Line = fiona.open('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb', 'r', layer='NHDLine')
NHDArea = fiona.open('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb', 'r', layer='NHDArea')

Flowline[1]

In [ ]:
NHDPLusEROMMA = fiona.open('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb', 'r', layer='NHDPlusEROMMA')

In [ ]:
import math

f_vector = lambda p,q: [q[0] - p[0], q[1] - p[1],  q[2] - p[2]]
f_distance = lambda p,q: np.sqrt ((p[0] - q[0])**2 + (p[1] - q[1])**2 + (p[2] - q[2])**2)

f_normal = lambda v,d: [v[0]/d, v[1]/d, v[2]/d]

def get_normal(coords):
    pos_v = f_vector(coords[0], coords[1])
    distance = math.sqrt(pos_v[0]**2 + pos_v[1]**2 + pos_v[2]**2)
    normal = f_normal(pos_v, distance)
    normal = [round(n,6) for n in normal]
    return normal

In [ ]:
km_to_m = 1000.
def River_stream_length():
    d2 = []
    for r in river:
        d1 = [('NHDPlusID',r.properties['NHDPlusID']), 
              ('LengthKM',r.properties['LengthKM']*km_to_m)]
        d2.append(d1)
    
    d3 = dict()
    d3['stream_lengthKM'] = d2
    return d3

In [ ]:
DataLength = River_stream_length()

In [ ]:
#QAMA : Flow from runoff, MA: Mean Annual
#VANA : Velocity for QAMA
#TotDASqKm: TotalDrainageAreaSqKm [may not be what we want??]

def rating_curve(q_cfs,v_fs):
    b, f, m = 0.26, 0.40, 0.34
    w = x1 * q_cfs**b
    d = x2 * q_cfs**f
    v = x3 * q_cfs**m
    
import scipy.signal
import geopy.distance


def cell_lengths_and_centroids_simple(index,spacing='Uniform',x_spacing=25.):
    length = Flowline[index]['properties']['LengthKM']*1000
    xx = Flowline[index]['properties']['NHDPlusID']
    
    if (spacing == "Uniform"):
        d = int(length/x_spacing)
        dx = length/d # ensures all dx sum to the total length
        cells_spacing = np.round(np.arange(0,length+dx,dx),4)
        cells_length = cells_spacing[1:] # first cell entity is ignored, zero length
    
    ncells = len(cells_length) if len(cells_length) > 0 else 1
    
    sum1 = 0
    
    x1 = Flowline[index]['geometry']['coordinates'][0]
    ncoords = len(x1)
    
    
    c1 = int(ncoords/(ncells))
    remain = ncoords%ncells # within a stream, cell volumes can be non-uniform, "remain" cells have large volume
    
    p_index = []
    
    for i in range(ncells+1):
        if i < remain:
            #print (i)
            p_index.append(i*(c1+1)) 
        else:
            p_index.append(p_index[i-1] + c1)
    
    edges_coords = []
    for p in p_index:
        coord = Flowline[index]['geometry']['coordinates'][0][p]
        edges_coords.append(coord)
        
    centroids = []
    
    for i in range(len(edges_coords)-1,0,-1):
        #old cent = [0.5* (edges_coords[i][0] + edges_coords[i+1][0]), 0.5* (edges_coords[i][1] + edges_coords[i+1][1]), 0]
        cent = [0.5* (edges_coords[i-1][0] + edges_coords[i][0]), 0.5* (edges_coords[i-1][1] + edges_coords[i][1]), 0]


        centroids.append(cent)
    
    centroids = list(np.concatenate(centroids))
    cells_spacing = list(cells_spacing)
    cells_length1 = [cells_spacing[0],]*len(cells_spacing)
    assert (len(centroids) == 3*len(cells_length))
   
    return centroids, cells_length, cells_spacing, len(cells_length)

In [ ]:
DataFlow = dict()
discharge = []
velocity = []

Normal = []
segment_depth = []
segment_length = []
segment_width = []
segment_crossA = []

km_to_m = 1000.
cfs_to_ms = 0.028316847 # QAMA is in cfs, convert cfs to m/s
fs_to_ms = 0.3048

# get discharge, velocity, depth, and orientation

def River_Characteristics():
    d4 = []
    d5 = []
    
    for k,r in enumerate(river):
        
        for i in range(len(NHDPLusEROMMA)):
            if NHDPLusEROMMA[i+1]['properties']['NHDPlusID'] == r.properties['NHDPlusID']:
                
                id = r.properties['NHDPlusID']
                
                value_q = NHDPLusEROMMA[i+1]['properties']['QAMA']*cfs_to_ms
                discharge.append([('NHDPlusID',id), ('Discharge [m^3/s]',value_q)])
                
                value_v = NHDPLusEROMMA[i+1]['properties']['VAMA']*fs_to_ms
                velocity.append([('NHDPlusID',id), ('Velocity [m/s]',value_v)])
                
                length = r.properties['LengthKM']*km_to_m
                
                #c = rating_curve(value_q,value_v)
                #break
                cross_area = value_q/value_v#FlowlineVAA[i+1]['properties']['TotDASqKm']
                                
                
                width = 1.0 #area/length*km_to_m
                height = cross_area * fs_to_ms #value_q/(value_v * width) 
                
                segment_length.append([('NHDPlusID',id), ('segment length [m]', length)])
                segment_depth.append([('NHDPlusID',id), ('segment depth [m]', height)])
                segment_width.append([('NHDPlusID',id), ('segment width [m]', width)])
                segment_crossA.append([('NHDPlusID',id), ('segment width [m]', cross_area * fs_to_ms)])
                
                # orientation
                #if Flowline[i+1]['properties']['NHDPlusID'] == r.properties['NHDPlusID']:
                p = Flowline[i+1]['geometry']['coordinates'][0][0]
                q = Flowline[i+1]['geometry']['coordinates'][0][-1] #note coord are in reverse order in NHD 
                print (k, r.properties['NHDPlusID'],length)
                n = get_normal([p,q])
                Normal.append([n[0],n[1],n[2]])
                
                # get number of cells and centroids
                
                
                
                #coordy = Flowline[i+1]['geometry']['coordinates']
                d4.append([('NHDPlusID',id), ('Discharge [m^3/s]',value_q),
                           ('Velocity [m/s]',value_v), ('segment length [m]', length),
                           ('segment depth [m]', height),
                           ('segment width [m]', width), ('orientation', [n[0],n[1],n[2]] )])
    
        
        for j in range(len(Flowline)):
            if Flowline[j+1]['properties']['NHDPlusID'] == r.properties['NHDPlusID']:
                centroids, cell_lengths, cell_spacing, ncells = cell_lengths_and_centroids_simple(j+1)
                d5.append([('NHDPlusID',id), ('centroids', centroids),
                           ('cell lengths [m]', cell_lengths), ('cell spacing [m]', cell_spacing),
                           ('number of cells', ncells)])
    
    
    return d4, d5


In [ ]:
Data_NHD_ATS, DataCell_NHD_ATS  = River_Characteristics()

In [ ]:
"""
import scipy.signal
import geopy.distance

#coords_1 = (52.2296756, 21.0122287)
#coords_2 = (52.406374, 16.9251681)


#geodistance = lambda x,y: geopy.distance.vincenty(x,y).km
#geodistance = lambda x,y: geopy.distance.geodesic(x,y).km
geodistance = lambda x,y: geopy.distance.distance(x,y).km
#print (geopy.distance.vincenty(coords_1, coords_2).km)

#xx = geodistance(coords_1,coords_2)
#print (xx)
def cell_volume_and_centroids(index):
    length = Flowline[index]['properties']['LengthKM']*1000
    xx = Flowline[index]['properties']['NHDPlusID']
    print ('Inside: ', index, length, xx)
    d = int(length/25.)
    cells_length = np.linspace(0,length,d)
    
    ncells = len(cells_length) if len(cells_length) > 0 else 1
    
    sum1 = 0
    print (ncells)
    x1 = Flowline[index]['geometry']['coordinates'][0]
    Total_Dist = []
    Total_Dist.append(0)
    for i in range(len(x1)-1):
        #sum1 = sum1 + f_distance(x1[i+1],x1[i]) * 1000.
        #print (x1[:2])
        sum1 = sum1 + geodistance(x1[i+1][:2],x1[i][:2]) * 1000.
        Total_Dist.append(sum1)
    
    j = 0
    edges_coords = []
    print ('Total dist, cell len:' ,len(Total_Dist), len(cells_length), Total_Dist[-1], cells_length[-1])
    for i , t in enumerate(Total_Dist):
        if t > cells_length[-1]:
            break
        #print (i,j)
        if t == cells_length[j]:
            coord = Flowline[index]['geometry']['coordinates'][0][i]
            edges_coords.append(coord)
            j = j + 1
        if t > cells_length[j]:
            left = abs(cells_length[j] - Total_Dist[i-1])
            right = abs(cells_length[j] - Total_Dist[i])
            if left <= right:
                coord = Flowline[index]['geometry']['coordinates'][0][i-1]
                edges_coords.append(coord)
            elif left > right:
                coord = Flowline[index]['geometry']['coordinates'][0][i]
                edges_coords.append(coord)
            j = j + 1
            
    centroids = []
    for i in range(len(edges_coords)-1):
        
        cent = [0.5* (edges_coords[i][0] + edges_coords[i+1][0]), 0.5* (edges_coords[i][1] + edges_coords[i+1][1]), 0]
        centroids.append(cent)
   
    centroids = list(np.concatenate(centroids))

    
    return centroids, cells_length[1:], len(cells_length[1:]) # first cell entity is ignored, zero length
"""

In [ ]:
def Streams_connectivity():
    Node = []
    for r in river:
        for i in range(len(FlowlineVAA)):
            if FlowlineVAA[i+1]['properties']['NHDPlusID'] == r.properties['NHDPlusID']:
                id = r.properties['NHDPlusID']
                from_node = FlowlineVAA[i+1]['properties']['FromNode']
                to_node = FlowlineVAA[i+1]['properties']['ToNode']
                
                Node.append([('NHDPlusID',id), ('FromNode',from_node),('ToNode',to_node)])
    Nodes = dict()
    Nodes['connectivity'] = Node
    return Nodes

In [ ]:
Connectivity = dict()
Connectivity_Orig = Streams_connectivity()

In [ ]:
import networkx as nx
pairs = [(1, 2),(2, 3),(3, 4), (3, 5),(5, 9),(5, 7), (2, 8), (8,10), (8,11)]

graph_syn = nx.from_edgelist(pairs)
print (graph_syn.nodes)



In [ ]:
coweeta_pairs = []

NHD_Id_Index = []
for i,n in enumerate(Connectivity_Orig['connectivity']):
    NHD_Id_Index.append([n[0][1],i+1])
    coweeta_pairs.append((int(n[2][1]),int(n[1][1])))
Coweeta_Graph = nx.from_edgelist(coweeta_pairs)

coweeta_pairs_index = []
node_index_pair = []

for i,node in enumerate(Coweeta_Graph):
    node_index_pair.append((node,i+1))

def get_index_from_node(node):
    for n in node_index_pair:
        if node == n[0]:
            #print ('from node', node,n)
            break
    return n[1]

def get_node_from_index(index):
    for n in node_index_pair:
        if index == n[1]:
            break
    return n[0]

for n in Connectivity_Orig['connectivity']:
    x1 = get_index_from_node(int(n[2][1]))
    x2 = get_index_from_node(int(n[1][1]))
    coweeta_pairs_index.append((x1,x2))
    
Coweeta_Graph1 = nx.from_edgelist(coweeta_pairs_index)


In [ ]:
def treeDiagnostic(graph):
    for node in graph:
        edges,_ = node_get_upstream_edges(graph,[node])
        for edge in edges:
            print (edge)
            assert edge[0] < edge[1]

def numNodes(graph):
    return len(graph.nodes)


def isNeighborNode(graph,node):
    ngh_nodes = list(graph.neighbors(node))
    return  True if len(ngh_nodes) > 0 else False

def node_get_edges(graph, node):
    
    edges = list(nx.edge_boundary(graph, node))
    return edges

def node_get_upstream_edges(graph, node):
    edges = list(nx.edge_boundary(graph, node))
    parent = -100
    for i,edge in enumerate(edges):
        if edge[0] > edge[1]:
            parent = edges[i]
            edges.pop(i)
    return edges, parent

def isNeighborBoundaryNode(graph, edge):
    ngh = list(graph.neighbors(edge[1]))
    return False if len(ngh) > 1 else True

def isBoundaryNode(graph, node):
    assert (node != 1)
    ngh = list(graph.neighbors(node))
    return False if len(ngh) > 1 else True

def get_node_index(graph, node):
    x = np.where(np.array(graph.nodes) == node)[0][0] + 1
    return x

def index_based_graph(graph):
    Nodes_new_ids = np.linspace(1,len(graph),len(graph), dtype='i')
    Nodes = graph.nodes


    

In [ ]:
from lxml import etree
import lxml.builder
import lxml.etree as ET

num_nodes = numNodes(Coweeta_Graph1)
fine_roots = []

Discharge =  []
CrossSectionArea = []
BoundaryCells = []
BoundaryReachCells = []
def WriteXMLNHD_Centroid():
    index = 0
    root = ET.Element("ParameterList", name="Main")
    ET.SubElement(root,"Parameter",name="infer cell centroids",type="bool", value="true")
    segments = ET.SubElement(root,"ParameterList", name="segments")
    nam = ''
    node_id_root = 0
    total_cells = 0
    bc_cell_id = 0
    tt = 0
    for i,node in enumerate(Coweeta_Graph1):
        
        length = Data_NHD_ATS[i][3][1]
        width = 1#Data_NHD_ATS[i][5][1]
        orientation = (1,0,0)#Data_NHD_ATS[i][6][1]
        cross_area = Data_NHD_ATS[i][4][1]
        
        centroids = DataCell_NHD_ATS[i][1][1]
        cell_lengths = DataCell_NHD_ATS[i][2][1]
        ncells = DataCell_NHD_ATS[i][4][1]
        
        total_cells = total_cells + ncells
        
        if i ==0:
            index = 0
            node_id_root = node
                       
            edges = node_get_edges(Coweeta_Graph1,[node])[0]
            
            nam = 'stream_' + str(edges[0]) + '_' + str(edges[1])
           
            streams = ET.SubElement(segments,"ParameterList", name="%s"%nam)
            ET.SubElement(streams, "Parameter", name="segment length [m]", type="double", value="%s"%length)
            
            ET.SubElement(streams,"Parameter",name="number of cells", type="int", value="%s"%ncells)
            
            n1,n2 = orientation[0],orientation[1]
            
            ET.SubElement(streams,"Parameter",name="orientation", type="Array(double)", value="{%s,%s,0}"%(n1,n2))
            ET.SubElement(streams,"Parameter",name="cross sectional area [m^2]", type="double", value="%s"%cross_area)
            ET.SubElement(streams,"Parameter",name="first tip type", type="string", value="boundary")
            ET.SubElement(streams,"Parameter",name="last tip type", type="string", value="junction")
            ET.SubElement(streams,"Parameter",name="cell centroids", type="Array(double)", value="%s"%centroids)
            ET.SubElement(streams,"Parameter",name="cell lengths [m]", type="Array(double)", value="%s"% cell_lengths)
            
            Discharge.append(Data_NHD_ATS[index][1][1])
            print (Discharge)
            break
            CrossSectionArea.append(Data_NHD_ATS[index][4][1])
            bc_cell_id = bc_cell_id + ncells
            tt  = tt + ncells
        else:
            edges, parent = node_get_upstream_edges(Coweeta_Graph1,[node])
           
            edge_len = len(edges)
            
            if edge_len == 0: #boundary edge
                continue
                
            for j, child_node in enumerate(edges):
                
                if child_node[1] != node_id_root:
                    index = child_node[1] - 2 
                    index_parent = child_node[0] - 2 
                    nam = 'stream_' + str(child_node[0]) + '_' + str(child_node[1])
                    streams = ET.SubElement(segments,"ParameterList", name="%s"%nam)
                   
                    centroids = DataCell_NHD_ATS[index][1][1]
                    cell_lengths = DataCell_NHD_ATS[index][2][1]
                    ncells = DataCell_NHD_ATS[index][4][1]
                    cross_area = Data_NHD_ATS[index][4][1]
                    length = Data_NHD_ATS[index][3][1]
                    
                    ET.SubElement(streams,"Parameter",name="first tip type", type="string", value="branch")
                    ET.SubElement(streams,"Parameter",name="first tip branch segment", type="string", value="stream_%s_%s"%(parent[1],parent[0]))
                    ET.SubElement(streams,"Parameter",name="first tip branch segment tip", type="string", value="last")
                    
                    if isBoundaryNode(Coweeta_Graph1,child_node[1]):
                        ET.SubElement(streams,"Parameter",name="last tip type", type="string", value="boundary")
                    else:
                        ET.SubElement(streams,"Parameter",name="last tip type", type="string", value="junction")
                        
                    n1,n2 = orientation[0],orientation[1]
                    
                    ET.SubElement(streams,"Parameter",name="orientation", type="Array(double)", value="{%s,%s,0}"%(n1,n2))
                    ET.SubElement(streams, "Parameter", name="segment length [m]", type="double", value="%s"%length)
                    ET.SubElement(streams,"Parameter",name="number of cells", type="int", value="%s"%ncells) 
                    ET.SubElement(streams,"Parameter",name="cross sectional area [m^2]", type="double", value="%s"%cross_area)
                    ET.SubElement(streams,"Parameter",name="cell centroids", type="Array(double)", value="%s"%centroids)
                    ET.SubElement(streams,"Parameter",name="cell lengths [m]", type="Array(double)", value="%s"% cell_lengths)
                    
                    fine_roots.append(nam)
                    Discharge.append(Data_NHD_ATS[index][1][1])
                    CrossSectionArea.append(Data_NHD_ATS[index][4][1])
                    print ()
                    bc_cell_id = bc_cell_id + ncells
                    # get boundary nodes ids
                    tt  = tt + ncells
                    if isBoundaryNode(Coweeta_Graph1,child_node[1]):
                        
                        BoundaryCells.append(bc_cell_id-1)
                        r_ids = [bc_cell_id -1 - i1 for i1 in range(ncells)] 
                        BoundaryReachCells.append(r_ids)
                        print (nam, Discharge[-1])
                    i = i + j
                #
            #
        #
        
        if (i+2 >= len(Coweeta_Graph1)):   
            break
    sets = ET.SubElement(root,"ParameterList",name="sets",type="bool", value="true")
    ET.SubElement(sets,"Parameter",name="fine_root",type="Array(string)", value="%s"%fine_roots)
    Data= etree.tostring(root,encoding="unicode",pretty_print=True)
    
    #outfile= open('coweeta_river-centriod1.xml','w')
    #outfile.write(Data)




In [ ]:
WriteXMLNHD_Centroid()

In [ ]:
BoundaryCells


In [ ]:
0.061198907756493/len(BoundaryReachCells[8])
# S_i = dQ/N * Co, dQ = Qout - Qin, N=len(reach), Co = concentrate

In [ ]:
Discharge =  []
CrossSectionArea = []
BoundaryCells = []

"""
def ATS_Ids_map():
    ats_nhd_id_map = []
    count1= 0
    for i,node in enumerate(Coweeta_Graph1):
        edges, parent = node_get_upstream_edges(Coweeta_Graph1,[node])
        
        if (parent == -100):
            continue
        index_parent = parent[0] - 2
        for j,edg in enumerate(edges):
            index1 = edg[1]-2
            count1 = count1 + 1
            ats_nhd_id_map.append([count1, index1])
            
            
            #print (index_parent, index1, ncount)
    return ats_nhd_id_map

print(list(nx.bfs_edges(Coweeta_Graph1,1)))
#print (map_ids)
def tree_shape():
    C = []
    for i,node in enumerate(Coweeta_Graph1):
        edges, parent = node_get_upstream_edges(Coweeta_Graph1,[node])
        if (parent == -100):
            continue
        index_parent = parent[0] - 2
        ncells = DataCell_NHD_ATS[index_parent][3][1]
        #for j,edg in enumerate(edges):
        C.append(ncells)
        print (C)
#tree_shape()
def get_map_id(index_parent):
    map_ids = ATS_Ids_map()
    print (map_ids)
"""

def Linearize_Discharge():
    QQ = []
    index = 0
    count = 0
    
    total_cells = 0
    map_id = []
    map_id.append(0)
    for i,node in enumerate(Coweeta_Graph1):
        if i ==-1:
            index = 0
            node_id_root = node
            Discharge.append(Data_NHD_ATS[index][1][1])
            
        else:
            edges, parent = node_get_upstream_edges(Coweeta_Graph1,[node])
            edge_len = len(edges)
            
            if (parent == -100):
                continue
            else:
                index_parent = parent[0] - 2
            
            #summing Qin
            #------------------------------------------------
            Qin = 0
            for edg in edges:
                index1 = edg[1]-2
                Qin = Qin + Data_NHD_ATS[index1][1][1] # Discharge
                
                count = count + 1
                map_id.append(index1)
            Boundary_Node = False
            if (edge_len==0): # boundary node
                Qin = 0
                Boundary_Node = True
                
            #------------------------------------------------
            Qout=Data_NHD_ATS[index_parent][1][1] # Dischage
            
            
            length = Data_NHD_ATS[index_parent][3][1] 
            centroids = DataCell_NHD_ATS[index_parent][1][1]
            cell_spacing = DataCell_NHD_ATS[index_parent][3][1]
            ncells = DataCell_NHD_ATS[index_parent][4][1]
            
            m = (Qout - Qin)/length
            
            
            #flux_f = [round(Qin + m*x,10) for x in cell_spacing[1:]]
            flux_f = [round(Qin + m*x,10) for x in cell_spacing]
            #flux_f[0] = flux_f[1]
            #assert (ncells == len(flux_f)-1)
            total_cells = total_cells + ncells
            
            if not Boundary_Node:
                flux_f = flux_f[1:]
            print (Qout, Qin, length, ncells, len(flux_f))
            QQ.append(flux_f[::-1])
            #if (edge_len==0):
                #print (QQ[-1])
                #print (flux_f[0])
    
    Q2 = []
    map1 = list(nx.bfs_edges(Coweeta_Graph1,1))
    
    for i in range(0,len(QQ)): # to get the fluxes consistent with ATS faces
        Q2.append(QQ[map_id[i]])
        
    np.set_printoptions(suppress=True)
    Q1 = np.concatenate(Q2,axis=0)
    print (len(Q1),total_cells)
    np.savetxt("/Users/ajc/Core/PostProcessData/2019/simulations/transport/flux_interpolate2.dat", Q1)
    


In [ ]:
Linearize_Discharge()